# Deploy Trained Smolvla Policy

<img src="./media/rollout3.gif" width="480" height="360">

Deploy trained policy in simulation.
# ========================================
# 8.smolvla.ipynb - SmolVLA策略部署与测试
# ========================================
# 功能：加载训练好的SmolVLA模型，并在MuJoCo仿真环境中进行实际部署测试
# SmolVLA = Small Vision-Language-Action Model（小型视觉-语言-动作模型）

In [ ]:
# Cell 1 - 设置环境变量(必须第一个运行)
import os

# 1. 设置DISPLAY
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 2. 强制使用GPU渲染(关键!)
os.environ['MUJOCO_GL'] = 'egl'  # EGL后端GPU加速
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# 3. NVIDIA GPU优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'  # 关闭垂直同步
os.environ['__GL_YIELD'] = 'NOTHING'      # 减少CPU等待
print("✓ NVIDIA GPU优化已启用")

# 4. OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'        # 关闭抗锯齿
os.environ['__GL_LOG_MAX_ANISO'] = '0'    # 关闭各向异性过滤
print("✓ OpenGL性能优化已启用")

In [ ]:
!pip install transformers==4.50.3
!pip install num2words
!pip install accelerate
!pip install safetensors>=0.4.3

### [Optional] Download Dataset

In [ ]:
'''
If you want to use the collected dataset, please download it from Hugging Face.
'''
#!git clone https://huggingface.co/datasets/Jeongeun/omy_pnp_language

In [ ]:
!huggingface-cli download Jeongeun/omy_pnp_language --repo-type dataset --local-dir ./demo_data_language


## Step 2. Train Model

In [ ]:
#!python train_model.py --config_path smolvla_omy.yaml
from train_with_monitor_cell import train_with_monitor
train_with_monitor("smolvla_omy.yaml")

## Step 3. Deploy

In [1]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
import numpy as np
from lerobot.common.datasets.utils import write_json, serialize_dict
from lerobot.common.policies.smolvla.configuration_smolvla import SmolVLAConfig
from lerobot.common.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.configs.types import FeatureType
from lerobot.common.datasets.factory import resolve_delta_timestamps
from lerobot.common.datasets.utils import dataset_to_policy_features
import torch
from PIL import Image
import torchvision

/home/wzy/envs/lerobot-mujoco/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' 

In [3]:
try:
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./demo_data_language')
except:
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./omy_pnp_language')
features = dataset_to_policy_features(dataset_metadata.features)
output_features = {key: ft for key, ft in features.items() if ft.type is FeatureType.ACTION}
input_features = {key: ft for key, ft in features.items() if key not in output_features}
# Policies are initialized with a configuration class, in this case `DiffusionConfig`. For this example,
# we'll just use the defaults and so no arguments other than input/output features need to be passed.
# Temporal ensemble to make smoother trajectory predictions
cfg = SmolVLAConfig(input_features=input_features, output_features=output_features, chunk_size= 5, n_action_steps=5)
delta_timestamps = resolve_delta_timestamps(cfg, dataset_metadata)

In [4]:
# We can now instantiate our policy with this config and the dataset stats.
policy = SmolVLAPolicy.from_pretrained('ckpt/smolvla_omy4060_20251103_224510/checkpoints/last/pretrained_model', dataset_stats=dataset_metadata.stats)
# You can load the trained policy from hub if you don't have the resources to train it.
# policy = SmolVLAPolicy.from_pretrained("Jeongeun/omy_pnp_pi0", config=cfg, dataset_stats=dataset_metadata.stats)
policy.to(device)



Reducing the number of VLM layers to 16 ...
Loading weights from local directory


SmolVLAPolicy(
  (normalize_inputs): Normalize(
    (buffer_observation_state): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 6 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 6 (cuda:0)]
    )
  )
  (normalize_targets): Normalize(
    (buffer_action): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
    )
  )
  (unnormalize_outputs): Unnormalize(
    (buffer_action): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
    )
  )
  (model): VLAFlowMatching(
    (vlm_with_expert): SmolVLMWithExpertModel(
      (vlm): SmolVLMForConditionalGeneration(
        (model): SmolVLMModel(
          (vision_model): SmolVLMVisionTransformer(
            (embeddings): SmolVLMVisio

In [6]:
# Cell 1 - 设置环境变量(必须第一个运行)
import os

# 1. 设置DISPLAY
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 2. 强制使用GPU渲染(关键!)
os.environ['MUJOCO_GL'] = 'egl'  # EGL后端GPU加速
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# 3. NVIDIA GPU优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'  # 关闭垂直同步
os.environ['__GL_YIELD'] = 'NOTHING'      # 减少CPU等待
print("✓ NVIDIA GPU优化已启用")

# 4. OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'        # 关闭抗锯齿
os.environ['__GL_LOG_MAX_ANISO'] = '0'    # 关闭各向异性过滤
print("✓ OpenGL性能优化已启用")

✓ DISPLAY设置为: :0
✓ MUJOCO_GL: egl (GPU硬件加速)
✓ NVIDIA GPU优化已启用
✓ OpenGL性能优化已启用


In [7]:
from mujoco_env.y_env2 import SimpleEnv2
xml_path = './asset/example_scene_y2.xml'
PnPEnv = SimpleEnv2(xml_path, action_type='joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[23]
 [0/23] [world] mass:[0.00]kg
 [1/23] [front_object_table] mass:[1.00]kg
 [2/23] [camera] mass:[0.00]kg
 [3/23] [camera2] mass:[0.00]kg
 [4/23] [camera3] mass:[0.00]kg
 [5/23] [link1] mass:[2.06]kg
 [6/23] [link2] mass:[3.68]kg
 [7/23] [link3] mass:[2.39]kg
 [8/23] [link4] mass:[1.40]kg
 [9/23] [link5] mass:[1.40]kg
 [10/23] [link6] mass:[0.65]kg
 [11/23] [camera_center] mass:[0.00]kg
 [12/23] [tcp_link] mass:[0.32]kg
 [13/23] [rh_p12_rn_r1] mass:[0.07]kg
 [14/23] [rh_p12_rn_r2] mass:[0.02]kg
 [15/23] [rh_p12_rn_l1] mass:[0.07]kg
 [16/23] [rh_p12_rn_l2] mass:[0.02]kg
 [17/23] [body_obj_mug_5] mass:[0.00]kg
 [18/23] [object_mug_5] mass:[0.08]kg
 [19/23] [body_obj_plate_11] mass:[0.00]kg
 [20/23] [object_plate_11] mass:[0.10]kg
 [21/23] [body_obj_mug_6] mass:[0.00]kg
 [22/23] [object_mug

In [8]:
from torchvision import transforms
# Approach 1: Using torchvision.transforms
def get_default_transform(image_size: int = 224):
    """
    Returns a torchvision transform that:
     Converts to a FloatTensor and scales pixel values [0,255] -> [0.0,1.0]
    """
    return transforms.Compose([
        transforms.ToTensor(),  # PIL [0–255] -> FloatTensor [0.0–1.0], shape C×H×W
    ])

In [10]:
# ========================================================================
# SmolVLA Policy Evaluation with Timeout and Success Rate Statistics
# SmolVLA策略评估脚本（带超时机制和成功率统计）
# ========================================================================

import time
import numpy as np
from datetime import datetime
from collections import defaultdict

# ========================================================================
# Configuration | 配置参数
# ========================================================================

NUM_EPISODES = 100        # Total number of episodes to test | 测试回合总数
TIMEOUT_SECONDS = 30      # Timeout for each episode (seconds) | 每回合超时时间（秒）
CONTROL_FREQUENCY = 20    # Control frequency (Hz) | 控制频率（赫兹）
MAX_STEPS_PER_EPISODE = TIMEOUT_SECONDS * CONTROL_FREQUENCY  # 最大步数 = 30秒 × 20Hz = 600步

# ========================================================================
# Statistics Container | 统计数据容器
# ========================================================================

class EvaluationStats:
    """
    Container for evaluation statistics
    评估统计数据容器
    """
    def __init__(self):
        self.total_episodes = 0          # 总回合数
        self.successful_episodes = 0     # 成功回合数
        self.failed_episodes = 0         # 失败回合数
        self.timeout_episodes = 0        # 超时回合数
        self.episode_durations = []      # 每回合耗时列表
        self.episode_steps = []          # 每回合步数列表
        self.success_steps = []          # 成功回合的步数
        
    def add_episode(self, success, timeout, duration, steps):
        """
        Add episode result to statistics
        添加回合结果到统计数据
        
        Args:
            success: Whether the episode succeeded | 是否成功
            timeout: Whether the episode timed out | 是否超时
            duration: Episode duration in seconds | 回合耗时（秒）
            steps: Number of steps taken | 执行的步数
        """
        self.total_episodes += 1
        self.episode_durations.append(duration)
        self.episode_steps.append(steps)
        
        if success:
            self.successful_episodes += 1
            self.success_steps.append(steps)
        elif timeout:
            self.timeout_episodes += 1
            self.failed_episodes += 1
        else:
            self.failed_episodes += 1
    
    def get_success_rate(self):
        """Calculate success rate | 计算成功率"""
        if self.total_episodes == 0:
            return 0.0
        return (self.successful_episodes / self.total_episodes) * 100
    
    def print_summary(self):
        """
        Print evaluation summary
        打印评估总结
        """
        print("\n" + "="*80)
        print("📊 EVALUATION SUMMARY | 评估总结")
        print("="*80)
        
        print(f"\n📈 Overall Statistics | 总体统计:")
        print(f"  Total Episodes | 总回合数:        {self.total_episodes}")
        print(f"  Successful | 成功:               {self.successful_episodes}")
        print(f"  Failed | 失败:                   {self.failed_episodes}")
        print(f"  Timeout | 超时:                  {self.timeout_episodes}")
        print(f"  Success Rate | 成功率:           {self.get_success_rate():.2f}%")
        
        if self.episode_durations:
            print(f"\n⏱️  Time Statistics | 时间统计:")
            print(f"  Average Duration | 平均耗时:    {np.mean(self.episode_durations):.2f}s")
            print(f"  Min Duration | 最短耗时:        {np.min(self.episode_durations):.2f}s")
            print(f"  Max Duration | 最长耗时:        {np.max(self.episode_durations):.2f}s")
        
        if self.episode_steps:
            print(f"\n🎯 Step Statistics | 步数统计:")
            print(f"  Average Steps | 平均步数:       {np.mean(self.episode_steps):.1f}")
            print(f"  Min Steps | 最少步数:           {np.min(self.episode_steps)}")
            print(f"  Max Steps | 最多步数:           {np.max(self.episode_steps)}")
        
        if self.success_steps:
            print(f"\n✅ Success Statistics | 成功回合统计:")
            print(f"  Avg Steps (Success) | 成功平均步数: {np.mean(self.success_steps):.1f}")
            print(f"  Min Steps (Success) | 成功最少步数: {np.min(self.success_steps)}")
            print(f"  Max Steps (Success) | 成功最多步数: {np.max(self.success_steps)}")
        
        print("="*80 + "\n")

# ========================================================================
# Main Evaluation Loop | 主评估循环
# ========================================================================

def evaluate_policy_with_timeout():
    """
    Evaluate policy performance with timeout mechanism
    评估策略性能（带超时机制）
    """
    # 初始化统计对象 Initialize statistics object
    stats = EvaluationStats()
    
    # 记录开始时间 Record start time
    evaluation_start_time = datetime.now()
    print("="*80)
    print("🚀 STARTING POLICY EVALUATION | 开始策略评估")
    print("="*80)
    print(f"Start Time | 开始时间: {evaluation_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total Episodes | 总回合数: {NUM_EPISODES}")
    print(f"Timeout | 超时时间: {TIMEOUT_SECONDS}s")
    print(f"Control Frequency | 控制频率: {CONTROL_FREQUENCY}Hz")
    print("="*80 + "\n")
    
    # ====================================================================
    # Episode Loop | 回合循环
    # ====================================================================
    for episode_idx in range(NUM_EPISODES):
        # 重置环境和策略 Reset environment and policy
        PnPEnv.reset(seed=episode_idx)  # 使用不同的seed确保多样性 Use different seed for diversity
        policy.reset()
        policy.eval()
        
        # 初始化回合变量 Initialize episode variables
        step = 0                           # 当前步数计数器 Current step counter
        episode_start_time = time.time()   # 回合开始时间 Episode start time
        episode_success = False            # 成功标志 Success flag
        episode_timeout = False            # 超时标志 Timeout flag
        
        print(f"Episode {episode_idx + 1}/{NUM_EPISODES} started... | 回合 {episode_idx + 1}/{NUM_EPISODES} 开始...", end=" ")
        
        # ================================================================
        # Step Loop | 步骤循环
        # ================================================================
        while PnPEnv.env.is_viewer_alive():
            # 推进物理仿真 Advance physics simulation
            PnPEnv.step_env()
            
            # 按控制频率执行（20Hz = 每秒20次）
            # Execute at control frequency (20Hz = 20 times per second)
            if PnPEnv.env.loop_every(HZ=CONTROL_FREQUENCY):
                
                # ========================================================
                # Timeout Check | 超时检查
                # ========================================================
                current_time = time.time()
                elapsed_time = current_time - episode_start_time
                
                if elapsed_time > TIMEOUT_SECONDS:
                    episode_timeout = True
                    print(f"⏱️  TIMEOUT after {elapsed_time:.1f}s | 超时")
                    break
                
                # ========================================================
                # Success Check | 成功检查
                # ========================================================
                success = PnPEnv.check_success()
                if success:
                    episode_success = True
                    episode_duration = time.time() - episode_start_time
                    print(f"✅ SUCCESS in {episode_duration:.2f}s ({step} steps) | 成功")
                    break
                
                # ========================================================
                # State Observation | 状态观测
                # ========================================================
                # 获取机器人关节状态（前6个关节）
                # Get robot joint states (first 6 joints)
                state = PnPEnv.get_joint_state()[:6]
                
                # ========================================================
                # Image Observation | 图像观测
                # ========================================================
                # 获取两个摄像头的图像：第三人称视角 + 腕部视角
                # Get images from two cameras: third-person view + wrist view
                image, wrist_image = PnPEnv.grab_image()
                
                # 图像预处理：PIL格式 -> 调整大小 -> Tensor
                # Image preprocessing: PIL format -> Resize -> Tensor
                image = Image.fromarray(image)           # NumPy数组转PIL图像 NumPy array to PIL image
                image = image.resize((256, 256))         # 调整到256x256 Resize to 256x256
                image = IMG_TRANSFORM(image)             # 转为Tensor [0,1] Convert to Tensor [0,1]
                
                wrist_image = Image.fromarray(wrist_image)
                wrist_image = wrist_image.resize((256, 256))
                wrist_image = IMG_TRANSFORM(wrist_image)
                
                # ========================================================
                # Prepare Model Input | 准备模型输入
                # ========================================================
                data = {
                    # 关节状态：[1, 6] 张量 Joint state: [1, 6] tensor
                    'observation.state': torch.tensor([state]).to(device),
                    
                    # 第三人称图像：[1, 3, 256, 256] 张量 Third-person image: [1, 3, 256, 256] tensor
                    'observation.image': image.unsqueeze(0).to(device),
                    
                    # 腕部图像：[1, 3, 256, 256] 张量 Wrist image: [1, 3, 256, 256] tensor
                    'observation.wrist_image': wrist_image.unsqueeze(0).to(device),
                    
                    # 任务描述（自然语言）Task description (natural language)
                    'task': [PnPEnv.instruction],  # 例如："pick the mug" or "place the plate"
                }
                
                # ========================================================
                # Policy Inference | 策略推理
                # ========================================================
                # 模型预测动作（无梯度计算）
                # Model predicts action (no gradient computation)
                with torch.no_grad():
                    action = policy.select_action(data)  # 输出shape: [1, chunk_size, 7]
                    action = action[0, :7].cpu().numpy() # 取第一步动作: [7]
                
                # 动作维度说明 Action dimensions:
                # action[0:6] - 6个关节的目标角度或角度增量 6 joint angles or increments
                # action[6]   - 夹爪开合状态 Gripper open/close state
                
                # ========================================================
                # Execute Action | 执行动作
                # ========================================================
                _ = PnPEnv.step(action)
                
                # ========================================================
                # Render Visualization | 渲染可视化
                # ========================================================
                PnPEnv.render()
                
                # 步数递增 Increment step counter
                step += 1
                
                # 步数限制检查（防止无限循环）
                # Step limit check (prevent infinite loop)
                if step >= MAX_STEPS_PER_EPISODE:
                    episode_timeout = True
                    print(f"⏱️  MAX STEPS REACHED | 达到最大步数")
                    break
        
        # ================================================================
        # Episode End | 回合结束
        # ================================================================
        episode_duration = time.time() - episode_start_time
        
        # 记录统计数据 Record statistics
        stats.add_episode(
            success=episode_success,
            timeout=episode_timeout,
            duration=episode_duration,
            steps=step
        )
        
        # 每10回合打印进度 Print progress every 10 episodes
        if (episode_idx + 1) % 10 == 0:
            current_success_rate = stats.get_success_rate()
            print(f"\n📊 Progress: {episode_idx + 1}/{NUM_EPISODES} | "
                  f"Success Rate: {current_success_rate:.2f}% | "
                  f"成功率: {current_success_rate:.2f}%\n")
    
    # ====================================================================
    # Evaluation Complete | 评估完成
    # ====================================================================
    evaluation_end_time = datetime.now()
    total_duration = (evaluation_end_time - evaluation_start_time).total_seconds()
    
    print("\n" + "="*80)
    print("🎉 EVALUATION COMPLETED | 评估完成")
    print("="*80)
    print(f"End Time | 结束时间: {evaluation_end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total Duration | 总耗时: {total_duration:.2f}s ({total_duration/60:.2f} minutes)")
    print("="*80)
    
    # 打印详细统计 Print detailed statistics
    stats.print_summary()
    
    # 保存结果到文件 Save results to file
    save_evaluation_results(stats, evaluation_start_time, evaluation_end_time)
    
    return stats

# ========================================================================
# Save Results to File | 保存结果到文件
# ========================================================================

def save_evaluation_results(stats, start_time, end_time):
    """
    Save evaluation results to a text file
    保存评估结果到文本文件
    """
    timestamp = start_time.strftime("%Y%m%d_%H%M%S")
    filename = f"evaluation_results_{timestamp}.txt"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("SmolVLA Policy Evaluation Results\n")
        f.write("SmolVLA策略评估结果\n")
        f.write("="*80 + "\n\n")
        
        f.write(f"Start Time | 开始时间: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"End Time | 结束时间: {end_time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Duration | 总耗时: {(end_time - start_time).total_seconds():.2f}s\n\n")
        
        f.write(f"Total Episodes | 总回合数: {stats.total_episodes}\n")
        f.write(f"Successful | 成功: {stats.successful_episodes}\n")
        f.write(f"Failed | 失败: {stats.failed_episodes}\n")
        f.write(f"Timeout | 超时: {stats.timeout_episodes}\n")
        f.write(f"Success Rate | 成功率: {stats.get_success_rate():.2f}%\n\n")
        
        if stats.episode_durations:
            f.write(f"Average Duration | 平均耗时: {np.mean(stats.episode_durations):.2f}s\n")
            f.write(f"Average Steps | 平均步数: {np.mean(stats.episode_steps):.1f}\n")
        
        if stats.success_steps:
            f.write(f"Average Steps (Success) | 成功平均步数: {np.mean(stats.success_steps):.1f}\n")
    
    print(f"📝 Results saved to: {filename} | 结果已保存到: {filename}")

# ========================================================================
# Execute Evaluation | 执行评估
# ========================================================================

if __name__ == "__main__":
    # 运行评估 Run evaluation
    evaluation_stats = evaluate_policy_with_timeout()

🚀 STARTING POLICY EVALUATION | 开始策略评估
Start Time | 开始时间: 2025-11-04 12:00:03
Total Episodes | 总回合数: 100
Timeout | 超时时间: 30s
Control Frequency | 控制频率: 20Hz

DONE INITIALIZATION
Episode 1/100 started... | 回合 1/100 开始... ✅ SUCCESS in 8.98s (95 steps) | 成功
DONE INITIALIZATION
Episode 2/100 started... | 回合 2/100 开始... ✅ SUCCESS in 9.97s (101 steps) | 成功
DONE INITIALIZATION
Episode 3/100 started... | 回合 3/100 开始... ✅ SUCCESS in 15.38s (160 steps) | 成功
DONE INITIALIZATION
Episode 4/100 started... | 回合 4/100 开始... ✅ SUCCESS in 15.54s (156 steps) | 成功
DONE INITIALIZATION
Episode 5/100 started... | 回合 5/100 开始... ✅ SUCCESS in 9.28s (95 steps) | 成功
DONE INITIALIZATION
Episode 6/100 started... | 回合 6/100 开始... ✅ SUCCESS in 15.07s (152 steps) | 成功
DONE INITIALIZATION
Episode 7/100 started... | 回合 7/100 开始... ✅ SUCCESS in 10.03s (101 steps) | 成功
DONE INITIALIZATION
Episode 8/100 started... | 回合 8/100 开始... ✅ SUCCESS in 14.52s (150 steps) | 成功
DONE INITIALIZATION
Episode 9/100 started... | 回合 9/100 开

In [9]:
step = 0
PnPEnv.reset(seed=0)
policy.reset()
policy.eval()
save_image = True
IMG_TRANSFORM = get_default_transform()
while PnPEnv.env.is_viewer_alive():
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # Check if the task is completed
        success = PnPEnv.check_success()
        if success:
            print('Success')
            # Reset the environment and action queue
            policy.reset()
            PnPEnv.reset()
            step = 0
            save_image = False
        # Get the current state of the environment
        state = PnPEnv.get_joint_state()[:6]
        # Get the current image from the environment
        image, wirst_image = PnPEnv.grab_image()
        image = Image.fromarray(image)
        image = image.resize((256, 256))
        image = IMG_TRANSFORM(image)
        wrist_image = Image.fromarray(wirst_image)
        wrist_image = wrist_image.resize((256, 256))
        wrist_image = IMG_TRANSFORM(wrist_image)
        data = {
            'observation.state': torch.tensor([state]).to(device),
            'observation.image': image.unsqueeze(0).to(device),
            'observation.wrist_image': wrist_image.unsqueeze(0).to(device),
            'task': [PnPEnv.instruction],
        }
        # Select an action
        action = policy.select_action(data)
        action = action[0,:7].cpu().detach().numpy()
        # Take a step in the environment
        _ = PnPEnv.step(action)
        PnPEnv.render()
        step += 1
        success = PnPEnv.check_success()
        if success:
            print('Success')
            break

DONE INITIALIZATION


/tmp/ipykernel_2603290/3859105902.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  'observation.state': torch.tensor([state]).to(device),


Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION
Success
DONE INITIALIZATION


KeyboardInterrupt: 

In [ ]:
# policy.push_to_hub(
#     repo_id='Jeongeun/omy_pnp_smolvla',
#     commit_message='Add trained policy for PnP task',
# )

In [ ]:
PnPEnv.env.close_viewer()